In [1]:
import zipfile
import os


zip_file = '/content/homer_bart.zip'


with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content/homer_bart')


extracted_path = '/content/homer_bart'
contents = os.listdir(extracted_path)
contents




['Bart', 'Homer']

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Preprocess the images
def preprocess_images(image_folder, img_size=(64, 64)):
    images = []
    labels = []
    for label in os.listdir(image_folder):
        label_path = os.path.join(image_folder, label)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                image = load_img(image_path, target_size=img_size)
                image_array = img_to_array(image) / 255.0
                images.append(image_array)
                labels.append(0 if label == 'Homer' else 1)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

image_folder = '/content/homer_bart'
images, labels = preprocess_images(image_folder)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.1, random_state=42)


datagen_train = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


datagen_test = ImageDataGenerator()


model = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    BatchNormalization(momentum=0.9, epsilon=1e-5, scale=True, center=True),
    Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(momentum=0.9, epsilon=1e-5, scale=True, center=True),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),

    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


batch_size = 32
train_generator = datagen_train.flow(train_images, train_labels, batch_size=batch_size)
steps_per_epoch = train_images.shape[0] // batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=35,
    validation_data=(datagen_test.flow(test_images, test_labels)),
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(datagen_test.flow(test_images, test_labels))
print(f'Test accuracy: {test_accuracy:.4f}')




Epoch 1/35
7/7 [==============================] - 2s 105ms/step - loss: 0.7044 - accuracy: 0.5667 - val_loss: 0.7811 - val_accuracy: 0.5926
Epoch 2/35
7/7 [==============================] - 1s 74ms/step - loss: 0.5735 - accuracy: 0.6952 - val_loss: 0.5471 - val_accuracy: 0.6667
Epoch 3/35
7/7 [==============================] - 1s 78ms/step - loss: 0.6345 - accuracy: 0.6286 - val_loss: 0.5674 - val_accuracy: 0.8148
Epoch 4/35
7/7 [==============================] - 1s 71ms/step - loss: 0.5323 - accuracy: 0.7667 - val_loss: 0.6148 - val_accuracy: 0.7407
Epoch 5/35
7/7 [==============================] - 1s 72ms/step - loss: 0.5437 - accuracy: 0.7571 - val_loss: 0.5914 - val_accuracy: 0.7037
Epoch 6/35
7/7 [==============================] - 1s 72ms/step - loss: 0.5009 - accuracy: 0.7333 - val_loss: 0.5723 - val_accuracy: 0.7037
Epoch 7/35
7/7 [==============================] - 1s 67ms/step - loss: 0.5583 - accuracy: 0.7429 - val_loss: 0.5351 - val_accuracy: 0.7037
Epoch 8/35
7/7 [==========